In [1]:
#pip install Pillow
#pip install opencv-python
#pip install pyzbar
#pip install base45
#pip install cryptography
#pip install cose
#pip install cbor2

In [2]:
from pyzbar.pyzbar import decode
import cv2
import zlib
import base45
from base64 import b64decode, b64encode
import cbor2
import json
from urllib.request import urlopen
import urllib
import ssl
import OpenSSL.crypto

from cryptography import x509
from cryptography.utils import int_to_bytes
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.ec import EllipticCurvePublicKey
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicKey
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import ec

from cose.messages import CoseMessage
from cose.headers import KID, Algorithm
from cose.keys import CoseKey
from cose.keys.keytype import KtyEC2, KtyRSA
from cose.keys.keyparam import KpKty, KpAlg, EC2KpX, EC2KpY, EC2KpCurve, RSAKpE, RSAKpN
from cose.algorithms import Es256, EdDSA, Ps256
from cose.keys.curves import P256

from cryptography import x509
from cryptography.hazmat.backends import default_backend

import datetime

## Estrazione payload da QR Code

In [3]:
filename = "craxi.png"
img = cv2.imread(filename)
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
decoded = decode(gray_img)
payload = decoded[0].data[4:]
print(payload)

b'6BFOXN%TS3DH+M8.IAS0RTAN:2MCID:D4YIJ%CM5*KR$PB2PK.IMWEO.1FJAF4INO4*J8OX4W$C2VL*LA 43/IE%TE6UG+ZE V1+GOT*O4U79YLY1S7HOPC5NDO-B5ET42HP6DPE65%PD0DJ WJO*QJZILDB523G*S2U2V8TQEDK8CKY2KPC9DLH*SUEDL*S VD6/D0SS*RS%VD$MTA6UQ.CD8CNED1HCD*S9:D:XIBEIVG39.19ADZ3EBKDHOJGDBDIT1NJGIA+OJ:CI-L3ZJA/3CZIJFVA1RO5VA81K0ECM8CXVDC8C 1JI7JSTNCA7G6M0/Q*DPZOOB95526FINVHMIL6-Z7WT4TJP7T4/YQE:73X77/RF/M.P4Z/5JINQ+MN/Q19QE8Q4A7E:7LYPLTQSV8*DOPCR5T4C941QU %V 9M2EWYF6.45OX6N3W/$KV1TBOU%7LNKEAIBM-IKRF%8WJC901F$5HZFV+%D2 634K0:KZ6ML-FOMDFAST4JCMNA%PQVMR80.0U*0'


## Decodifica B45 e decompressione

In [4]:
decoded = base45.b45decode(payload)
decompressed = zlib.decompress(decoded)
print(decompressed)

b'\xd2\x84M\xa2\x04H\x1e\x19\x1e\xaa\xfa\xedCE\x01&\xa0Y\x01\x03\xa4\x04\x1adp\x9d\xa6\x06\x1aa\x82\xc5\xbc\x01bPL9\x01\x03\xa1\x01\xa4av\x81\xaabdn\x01bmamORG-100001417bvpgJ07BX03bdtj2021-10-01bcobPLbcix1URN:UVCI:01:PL:1/2A992C33754A4D379A7F61089485BB75bmplEU/1/20/1525bisqCentrum e-Zdrowiabsd\x01btgi840539006cnam\xa4cfnteCRAXIbfneCraxicgntgBETTINObgngBettinocvere1.0.0cdobj1934-02-24X@d!\xe1\x9f\x90\xeb\xf7\xf8\x94\x9bRj\xab\x06\x7f\xbebS\xa7]_Y\xca\xf9\n9;\x82v\xae\xe5\xcf\xdd4QkZ\xf0\xfe\xdfO={(6\xe7.d\x0f\xaf5\xe8\xea\xb4\xfd\xb2\xd2A\x0b\x067\rkc'


## Decodifica messaggio COSE

In [5]:
cose_message = CoseMessage.decode(decompressed)
algorithm = cose_message.get_attr(Algorithm).fullname
kid = cose_message.get_attr(KID)
cose_message

<COSE_Sign1: [{'KID': b'\x1e\x19\x1e\xaa\xfa\xedCE', 'Algorithm': 'Es256'}, {}, b'\xa4\x04\x1adp' ... (259 B), b'd!\xe1\x9f\x90' ... (64 B)]>

## Messaggio CBOR in payload COSE

In [6]:
#print(json.dumps(cbor2.loads(cose.payload), indent=2))

# from Tobias Girstmair (https://gir.st)
# https://gist.github.com/dsoares/dbd784615defd8800e93e4df4c783ce1

sch = urlopen('https://raw.githubusercontent.com/ehn-dcc-development/ehn-dcc-schema/release/1.3.0/DCC.combined-schema.json')
glb_schema = json.load(sch)

def annotate(data, schema, level=0):
    for key, value in data.items():
        description = schema[key].get('title') or schema[key].get('description') or key
        description, _, _ = description.partition(' - ')
        if type(value) is dict:
            print('  '*level, description)
            _, _, sch_ref = schema[key]['$ref'].rpartition('/')
            annotate(value, glb_schema['$defs'][sch_ref]['properties'], level+1)
        elif type(value) is list:
            print('  '*level, description)
            _, _, sch_ref = schema[key]['items']['$ref'].rpartition('/')
            for v in value:
                annotate(v, glb_schema['$defs'][sch_ref]['properties'], level+1)
        else: # value is scalar
            print('  '*level, description, ':', value)

data = cbor2.loads(cose_message.payload)
annotate(data[-260][1], glb_schema['properties'])

 Vaccination Group
   Dose Number : 1
   Marketing Authorization Holder : ORG-100001417
   vaccine or prophylaxis : J07BX03
   ISO8601 complete date: Date of Vaccination : 2021-10-01
   Country of Vaccination : PL
   Unique Certificate Identifier: UVCI : URN:UVCI:01:PL:1/2A992C33754A4D379A7F61089485BB75
   vaccine medicinal product : EU/1/20/1525
   Certificate Issuer : Centrum e-Zdrowia
   Total Series of Doses : 1
   disease or agent targeted : 840539006
 Surname(s), forename(s)
   Standardised surname : CRAXI
   Surname : Craxi
   Standardised forename : BETTINO
   Forename : Bettino
 Schema version : 1.0.0
 Date of birth : 1934-02-24


## Recupero chiavi pubbliche

In [7]:
url = 'https://get.dgc.gov.it/v1/dgc/signercertificate/update'
token = ''

certificates = {}
x509_collection = {}

while True:
    
    request = urllib.request.Request(url)
    if (token):
        request.add_header('X-RESUME-TOKEN', token)
        
    with urlopen(request) as keyfile:
        
        key = keyfile.read()
        if (not key): break
        token = keyfile.getheader('X-RESUME-TOKEN')
        b64kid = keyfile.getheader('X-KID')
        
        #print(keyfile.getheaders())
        
        deckey = b64decode(key)
        x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_ASN1, deckey)
        x509_collection[b64kid] = x509
        pk_data = x509.get_pubkey()
        
        pub_key = OpenSSL.crypto.dump_publickey(OpenSSL.crypto.FILETYPE_ASN1, pk_data)
        pub = serialization.load_der_public_key(pub_key)
        
        # key types: https://pycose.readthedocs.io/en/latest/cose/keys/keytype.html
        # key parameters: https://pycose.readthedocs.io/en/latest/cose/keys/keyparam.html
        if (isinstance(pub, EllipticCurvePublicKey)):
            # ECDSA SHA256
            # curves: https://pycose.readthedocs.io/en/latest/cose/keys/curves.html
            certificates[b64kid] = CoseKey.from_dict({
                KpKty: KtyEC2,
                KpAlg: Es256,
                EC2KpCurve: P256,
                EC2KpX: pub.public_numbers().x.to_bytes(32, byteorder="big"),
                EC2KpY: pub.public_numbers().y.to_bytes(32, byteorder="big")
            })    
        elif (isinstance(pub, RSAPublicKey)):
            # RSSA-PSS SHA-56 MFG1
            certificates[b64kid] = CoseKey.from_dict({
                KpKty: KtyRSA,
                KpAlg: Ps256, 
                RSAKpE: int_to_bytes(pub.public_numbers().e),
                RSAKpN: int_to_bytes(pub.public_numbers().n)
            })
        else:
              print(f"Tipo chiave sconosciuto {pub.__class__.__name__}).")          

In [8]:
# Trova chiave corretta per certificato
kid = b64encode(cose_message.get_attr(KID)).decode('ASCII')
pk_to_use = certificates[kid]

#Stampa attributi chiave trovata
print(f"KID: {kid}")
iss = x509_collection[kid].get_issuer().get_components()
c = [(x.decode('UTF-8'), y.decode('UTF-8')) for (x,y) in iss if x.decode('UTF-8') == "C"]
if(c): print(f"Country: {c[0][1]}")
cn = [(x.decode('UTF-8'), y.decode('UTF-8')) for (x,y) in iss if x.decode('UTF-8') == "CN"]
if(cn): print(f"Common name: {cn[0][1]}")
o = [(x.decode('UTF-8'), y.decode('UTF-8')) for (x,y) in iss if x.decode('UTF-8') == "O"]
if(o): print(f"Organization: {o[0][1]}")   

valid_from = datetime.datetime.strptime(x509_collection[kid].get_notBefore().decode('ASCII'),"%Y%m%d%H%M%SZ")
print(f"Valido da: {valid_from}")
valid_to = datetime.datetime.strptime(x509_collection[kid].get_notAfter().decode('ASCII'),"%Y%m%d%H%M%SZ")
print(f"Valido fino a: {valid_to}")

KID: HhkeqvrtQ0U=
Country: PL
Common name: Poland DGC RootCSCA 1
Organization: Ministry of Health
Valido da: 2021-05-26 11:53:10
Valido fino a: 2023-05-26 11:53:10


In [9]:
cose_message.key = pk_to_use
if not cose_message.verify_signature():
    print("Firma NON valida")
else:
    print("Firma valida")

Firma valida


## Verifica lista revoca chiavi

In [10]:
valid_url = 'https://get.dgc.gov.it/v1/dgc/signercertificate/status'
valid_list = []
with urlopen(valid_url) as valid_file:
    valid_list = valid_file.read()
    valid_list = json.loads(valid_list.decode('ASCII'))

if (kid not in valid_list):
    print("Certificato revocato")
else:
    print("Certificato valido")

Certificato valido


## Verifica lsta revoca certificati

In [12]:
settings_url = 'https://get.dgc.gov.it/v1/dgc/settings'
revoked_list =[]

with urlopen(settings_url) as settings_file:
    settings = settings_file.read()
    settings = json.loads(settings.decode('UTF-8'))
    
    revoked_data = [x for x in settings if x['name'] == 'black_list_uvci']    
    if (revoked_data): revoked_list = revoked_data[0]['value'].split(';')

gp_id = data[-260][1]['v'][0]['ci']

if (gp_id in revoked_list):
    print("Green Pass revocato")
else:
    print("Green Pass valido")

Green Pass revocato
